**二维阻尼振荡器**

作为第一个说明性示例，我们考虑具有三次动态的二维阻尼谐振子，即：

$$
\begin{array}{l}
\dot{x} = -0.1\ x^3 + 2.0\ y^3,\\
\dot{y} = -2.0\ x^3 - 0.1\ y^3.
\end{array}
$$

我们使用初始条件 $[x_0\ y_0]^T = [2\ 0]^T$，并从 $t = 0$ 到 $t = 25$ 收集数据，时间步长为 $\Delta t = 0.01$。

### 原始代码  
BsplineKANLayer

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
# from deepkan import SplineLinearLayer
from KAN import KANModel
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import nodepy.linear_multistep_method as lm
import torch.nn as nn
import torch.optim as optim
import timeit

torch.set_default_dtype(torch.float64) # 设置默认张量数据类型为双精度浮点数

# define solution
sol_fun_1 = lambda x:  (-0.1)* x[:,[0]]**3 + 2 * x[:,[1]]**3 
sol_fun_2 = lambda x:  (-2)* x[:,[0]]**3 + (-0.1) * x[:,[1]]**3 

start_time = timeit.default_timer()
# 超参数
n_steps = 10000            # 样本取点数               
steps = 5000             # 迭代次数
log = 1                  # 迭代序号
M = 1                    # 多步法的步数
y0 = np.array([0,1])     # 方程初始值
t_np = np.linspace(0, 10, num=n_steps)
dt = t_np[M] - t_np[0]   # 步长


def f(x,t): # x is 2 x 1
    A = np.array([[-.1,2], [-2,-.1]]) # 2 x 2
    f = np.matmul(A,x[:,None]**3) # 2 x 1
    return f.flatten()

# 计算出微分方程的数值解，它使用 lsoda 方法进行数值积分，lsoda 方法使用了 Gear 公式的变步长、变阶数实现。它是一种隐式的多步法
y_np = odeint(f, y0, t_np) 
y_train = y_np[0::M,:]
y_train_new  = torch.tensor(y_train)

lamb = 0.1
model = KANModel(width=[2,5,2],grid_range=[-4,4])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999), eps=1e-32)


def net_Y(alpha,beta,T,y_train_new):
    Y = alpha[0] * T[:, M:, :] + dt * beta[0] * model(y_train_new[M:, :]).unsqueeze(0)
    for m in range(1, M + 1):
        Y = Y + alpha[m] * T[:, M-m:-m, :] + dt * beta[m] * model(y_train_new[M-m:-m, :]).unsqueeze(0) # S x (N-M+1) x D
    return Y


# 用于存储每次迭代的损失值
Train_loss_list = []
Test_loss_list = []
iteration_list = []

start_time = timeit.default_timer()
for epoch in range(steps):
    model.train()
    scheme = 'AM'
    switch = {'AM': lm.Adams_Moulton,
                'AB': lm.Adams_Bashforth,
                'BDF': lm.backward_difference_formula}
    method = switch[scheme](M)

    # 将 method.alpha 和 method.beta 转换为浮点数数组
    alpha = torch.tensor(-np.array(method.alpha[::-1], dtype=np.float32))
    beta = torch.tensor(np.array(method.beta[::-1], dtype=np.float32))
    T = y_train_new.unsqueeze(0)
    optimizer.zero_grad()
    Y_pred = net_Y(alpha,beta,T,y_train_new)
    loss = criterion(Y_pred, torch.zeros_like(Y_pred)) 
    l2loss = criterion(Y_pred, torch.zeros_like(Y_pred)) + lamb * model.reg(y_train_new)
    # l2loss.backward()
    l2loss.backward(retain_graph=True)
    optimizer.step()

    # 测试误差
    test_steps = 1000
    test_np = np.linspace(10 ,20 , num=test_steps)
    y_test_np1 = odeint(f, y0, test_np)[:,0] 
    y_test_np2 = odeint(f, y0, test_np)[:,1]
    y_test_np = torch.stack([torch.from_numpy(y_test_np1), torch.from_numpy(y_test_np2)], axis=1)
    f_test_1 =sol_fun_1(y_test_np)
    f_test_2 =sol_fun_2(y_test_np)
    f_test = torch.cat((f_test_1, f_test_2), dim=1).detach()
    f_nn= model(y_test_np).detach()
    Test_loss =  torch.mean((f_test - f_nn)**2)

    # 存储损失值和迭代次数
    Train_loss_list.append(loss.cpu().detach().numpy())
    Test_loss_list.append(Test_loss.cpu().detach().numpy())
    iteration_list.append(epoch)

    if epoch % 500 == 0:
        print(f'Epoch {epoch+1}/{steps}, Train_loss: {loss.item(): .16e},  Test_loss: {Test_loss.item(): .16e}')
        # print(f'Epoch {epoch+1}/{steps}, loss: {loss.item(): .16e}')

elapsed = timeit.default_timer() - start_time
print(f"Time: {elapsed:.4f}")

In [ ]:
# Ploting
plt.figure(figsize=(10, 6))
plt.plot(iteration_list, Train_loss_list, label='Train loss', color='b')  # 绘制训练损失曲线，蓝色
plt.plot(iteration_list, Test_loss_list, label='Test loss', color='r')  # 绘制测试损失曲线，红色
plt.yscale('log')
# plt.ylim(10e-15, 10e1)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss vs. Iteration')
plt.legend()
plt.grid(True)
plt.show()
# plt.savefig('Cubic2D_Error_10-6.png', format='png', dpi=600)

In [ ]:
test_steps = n_steps
test_np = np.linspace(0, 10, test_steps)
test_np_new = torch.linspace(0, 10, test_steps)
# t_span = (0, 2)

def predict_f(y_star):
    y_star_tensor = torch.tensor(y_star)
    with torch.no_grad():
        F_star = model(y_star_tensor)
    return F_star.numpy()


def learned_f(x,t):
    f = predict_f(x[None,:])
    return f.flatten()

# def learned_f(t,x):
#     f = predict_f(x[None,:])
#     return f.flatten()

# learned_X_star = solve_ivp(learned_f, t_span, y0, method='BDF', t_eval=test_np)

learned_X_star = odeint(learned_f, y0, test_np)     
y_np1 = odeint(f, y0, test_np) 
# source_fun_1(test_np_new).type

lossx1 = np.max(learned_X_star[:,0]-y_np[:,0])
lossx2 = np.max(learned_X_star[:,1]-y_np[:,1])
print('x1_loss: {lossx1: .16e}, x2_loss: {lossx2: .16e}'.format(lossx1=lossx1, lossx2=lossx2))

In [ ]:
####### Plotting ################## 
#source_fun_1 = lambda t: 0.5*torch.exp(2*t) - 0.5*torch.exp(-4*t)
#source_fun_2 = lambda t: torch.exp(-4*t) 

plt.figure(figsize=(16, 9))
plt.plot(test_np, learned_X_star[:,0], label='learned_y_1',linestyle='--', color='r')
plt.plot(test_np, learned_X_star[:,1], label='learned_y_2',linestyle='--', color='b')
# plt.plot(test_np, learned_X_star.y.T[:,0], label='learned_y_1',linestyle='--', color='r')
# plt.plot(test_np, learned_X_star.y.T[:,1], label='learned_y_2',linestyle='--', color='b')
plt.plot(test_np, y_np1[:,0], label='y_1', color='g')
plt.plot(test_np, y_np1[:,1], label='y_2', color='c')
plt.axvline(x=10, color='k', linestyle='--', linewidth=2)
# plt.savefig('x_xnn10-20.png', format='png', dpi=600)

plt.legend()
plt.show

In [ ]:
####### Plotting ################## 
import matplotlib.gridspec as gridspec
from plotting import newfig, savefig

fig, ax = newfig(1.0, 0.9)
ax.axis('off')

gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=0.85, bottom=0.25, left=0.1, right=0.95, wspace=0.3)

ax = plt.subplot(gs0[:, 0:1])
ax.plot(test_np,y_np1[:,0],'m',label='$x$')
ax.plot(test_np,y_np1[:,1],'c',label='$y$')
ax.plot(test_np,learned_X_star[:,0],'k--',label='learned model')
ax.plot(test_np,learned_X_star[:,1],'k--')    
ax.set_xlabel('$t$')
ax.set_ylabel('$x, y$')
ax.legend(loc='upper center', bbox_to_anchor=(0.9, -0.25), ncol=3, frameon=False)
ax.set_title('Trajectories', fontsize = 10)

ax = plt.subplot(gs0[:, 1:2])
ax.plot(y_np1[:,0],y_np1[:,1], 'r', label='$(x,y)$')
ax.plot(learned_X_star[:,0],learned_X_star[:,1],'k--')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.legend(loc='upper center', bbox_to_anchor=(0.4, -0.25), ncol=1, frameon=False)
ax.set_title('Phase Portrait', fontsize = 10)


plt.savefig('Cubic2Dx_xnn25_0.005.png', format='png', dpi=600)
# savefig('./figures/Cubic2D')